In [1]:
#OLLAMAPool Server Prototype Code
import os
EndPoint_Queries=os.environ.get('EndPoint_Queries')
Endpoint_Results=os.environ.get('Endpoint_Results')
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_Queries is None:
    raise ValueError("EndPoint_Queries is not set")
if Endpoint_Results is None:
    raise ValueError("Endpoint_Results is not set")
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
print("Environment Variables are set OK")


Environment Variables are set OK


In [2]:
#Example Use case -> character description generator for fantasy RPG
#Takes a random dictionary of character traits and generates a written character description

races=["elf","human","dwarf","halfling","gnome"]
complections=["pale","dark","fair","ruddy","swarthy"]
heights=["tall","short","average"]
builds=["slender","stocky","muscular","portly"]
features=["sharp","soft","angular","round"]
eyes=["bright","dark","piercing","dull"]
hairs=["long","short","curly","straight","wavy"]
clothes=["fine","simple","elegant","gaudy","tattered"]
sophistication=["refined","crude","simple","complex"]
mannerisms=["graceful","clumsy","boisterous","quiet"]
virtues=["honest","loyal","brave","generous","kind"]


import random

#Come up with a random dictionary of character traits
def GenerateCharacterDescDictionary():
    traits={}
    race=random.choice(races)
    complection=random.choice(complections)
    height=random.choice(heights)
    build=random.choice(builds)
    feature=random.choice(features)
    eye=random.choice(eyes)
    hair=random.choice(hairs)
    clothe=random.choice(clothes)
    sophisticate=random.choice(sophistication)
    mannerism=random.choice(mannerisms)
    virtue=random.choice(virtues)
    traits={"race":race,"complection":complection,"height":height,"build":build,"feature":feature,"eye":eye,"hair":hair,"clothes":clothe,"sophistication":sophisticate,"mannerism":mannerism,"virtue":virtue}
    return traits

#Turn the dictionary into a line separated string
def GenerateCharacterDescList(traits):
    desc="They have the followng traits:\n"
    for key in traits:
        desc+=key+": "+traits[key]+"\n"
    return desc
    
            
    
#Test code
traits=GenerateCharacterDescDictionary()
print(GenerateCharacterDescList(traits))


They have the followng traits:
race: gnome
complection: swarthy
height: short
build: slender
feature: angular
eye: dark
hair: straight
clothes: fine
sophistication: simple
mannerism: clumsy
virtue: kind



In [3]:
#Produce sample character descriptions
totalRecords=10
requestTexts=[]
for i in range(totalRecords):
    traits=GenerateCharacterDescDictionary()
    requestTexts.append(GenerateCharacterDescList(traits))
requestTexts

['They have the followng traits:\nrace: gnome\ncomplection: ruddy\nheight: tall\nbuild: muscular\nfeature: sharp\neye: dull\nhair: curly\nclothes: elegant\nsophistication: refined\nmannerism: clumsy\nvirtue: brave\n',
 'They have the followng traits:\nrace: halfling\ncomplection: pale\nheight: tall\nbuild: muscular\nfeature: round\neye: bright\nhair: long\nclothes: fine\nsophistication: simple\nmannerism: graceful\nvirtue: brave\n',
 'They have the followng traits:\nrace: gnome\ncomplection: fair\nheight: average\nbuild: portly\nfeature: sharp\neye: dark\nhair: long\nclothes: simple\nsophistication: simple\nmannerism: clumsy\nvirtue: loyal\n',
 'They have the followng traits:\nrace: elf\ncomplection: dark\nheight: average\nbuild: slender\nfeature: sharp\neye: dark\nhair: long\nclothes: fine\nsophistication: refined\nmannerism: quiet\nvirtue: honest\n',
 'They have the followng traits:\nrace: human\ncomplection: pale\nheight: short\nbuild: muscular\nfeature: round\neye: piercing\nhair: 

In [4]:
#Busines Classes
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage

class LLMRequest:
    def __init__(self,Model:str="",systemMessage:str="",query:str=""):
        self.UUID=str(uuid.uuid4())
        self.Model=Model
        self.systemMessage=systemMessage
        self.query=query
        self.Messages=[{'role': 'system', 'content': systemMessage},
                       {'role': 'user', 'content': query}]

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMResult:
    def __init__(self,UUID:str="",result:str="",errorMsg:str="",timeDelta:str=""):
        self.UUID=UUID
        self.result=result
        self.errorMsg=errorMsg
        self.HasError=errorMsg!=""
        self.timeDelta=timeDelta

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMRequestResultPair():
    def __init__(self,request:LLMRequest,result:LLMResult=None):
        self.uuid=request.UUID
        self.request=request
        self.result=result


In [8]:
#Main OLLAMA Calling Function / Example code which posts data to queue
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage



#Post to a service bus queue    
def AzurePost_ServiceBus(connection_string: str, queue_name: str, json_payload):
    with ServiceBusClient.from_connection_string(connection_string) as client:
        sender = client.get_queue_sender(queue_name=queue_name)
        message = ServiceBusMessage(json.dumps(json_payload))
        with sender:
            sender.send_messages(message)
            print(f"Queued message: {json_payload['UUID']}")

#Take a list of queries and turn them into results
def BatchTransForm(Model:str,systemMessage:str,queries:List[str],DynamicUI:bool=True,WebUI:bool=False,ShowWebUI:bool=False)->List[str]:
    
    #Put every prompt into the LLM_Requests pool
    LLMrequests=[]
    for query in queries:
        request=LLMRequest(Model,systemMessage,query)
        LLMrequests.append(request)
        AzurePost_ServiceBus(EndPoint_Queries,"llm-queries",request.to_json())
    
    #Monitor the results queue
    return LLMrequests
    
    #Output the results
    
#Test code
systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""
queuedRequests=BatchTransForm("mistral-small",systemPrompt,requestTexts)



Queued message: cf71ff54-888d-4f90-a71d-2f3153371daa
Queued message: 12775d03-22c0-402a-b000-544ef8542af4
Queued message: a89011f0-e044-4381-8eb2-4cfc399f18f9
Queued message: 6808e544-475e-44d6-86b3-0fa0be810f9f
Queued message: 4c4dd04e-bc2b-4829-900d-374b8b87bd06
Queued message: 7b37483b-196d-4e41-a7ea-97dffd765685
Queued message: 8ab30491-e979-4a35-adae-b320b7bb4726
Queued message: 444958cc-6895-4e8f-a8e5-55fc47bfbf97
Queued message: a831677a-9383-4127-aa28-c3a7b4b9f089
Queued message: e9e865b5-be1f-424c-9db8-a14a92ffad12


In [7]:
#Main Result Handling Loop
import time
from typing import List
from azure.servicebus import ServiceBusClient, ServiceBusMessage

#Turn queud requests into dictionary
queueDict={}
for request in queuedRequests:
    queueDict[request.UUID]=LLMRequestResultPair(request)
totalResults=0
invalidResults=0

def ProcessResult(result:LLMResult):
    global totalResults
    global invalidResults
    if result.UUID in queueDict:
        print(f"Result UUID: {result.UUID} found in queueDict")
        queueDict[result.UUID].result=result
        totalResults+=1
    else:
        print(f"Result UUID: {result.UUID} not found in queueDict")
        invalidResults+=1

# Replace with your connection string and queue name
queue_name = "llm-responses"
servicebus_client = ServiceBusClient.from_connection_string(conn_str=Endpoint_Results)

def receive_messages_from_queue():
    # Create a receiver for the queue
    with servicebus_client.get_queue_receiver(queue_name=queue_name) as receiver:
        print("Receiving messages from the queue...")
        
        # Receive messages in a batch, you can specify max_message_count as needed
        received_msgs = receiver.receive_messages(max_message_count=10, max_wait_time=5)

        for msg in received_msgs:
            # Print the message payload
            print(f"Received message: {str(msg)}")
            receiver.complete_message(msg)
            
            llmRequest=LLMResult()
            llmRequest.from_json(str(msg))
            #llmRequest.from_json(json.loads(str(msg)))
            ProcessResult(llmRequest)
            
            # Accept the message to remove it from the queue
            #receiver.complete_message(msg)

# Call the function to read from the queue
while totalResults<len(queueDict):
    receive_messages_from_queue()
    print(f"Total Results: {totalResults}, Invalid Results: {invalidResults}")



Receiving messages from the queue...
Received message: {"UUID": "dcbac0e5-dbc0-42cf-ad3a-9f2285bc7d8c", "result": {"model": "llama3.1", "created_at": "2024-09-23T19:37:40.363504013Z", "message": {"role": "assistant", "content": "In the heart of the mystical forest of Elvendom, where ancient trees whispered secrets to the wind, and faeries danced with abandon under the silver light of the full moon, there lived a gnome like no other. His name was Bramble Burrowes, a towering figure among his diminutive kin, with a ruddy complexion that seemed to glow from within, as if infused by the warm embers of a thousand sunsets.\n\nStanding at an impressive seven feet tall, Bramble's muscular physique seemed to have been chiseled from the very stone of the mountains he called home. His broad shoulders and powerful arms spoke of a life spent in the service of his people, defending their underground strongholds against the dark forces that sought to claim the land for their own.\n\nBramble's sharp f

In [14]:
#Edge Case -> delibrately fail a few requests

systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""

#Invalid model!
queuedRequests=BatchTransForm("herdepr28.1",systemPrompt,requestTexts[0:1])



Queued message: fb9bc274-07af-4a99-9f37-8db0f5cbe309
